## Code to manually create tables for the cleaned dataset files

*   comments
*   follows
* likes
* photo_tags
* photos
* tags
* users


In [ ]:
import pandas as pd
import mysql.connector
from datetime import datetime

# MySQL database information
HOST = ''
USERNAME = ''
PASSWORD = ''
DATABASE = ''

# Creating a connection to the database
cnx = mysql.connector.connect(host=HOST, user=USERNAME, password=PASSWORD, database=DATABASE)
cursor = cnx.cursor()

# Droping the existing tables if they exist
cursor.execute("DROP TABLE IF EXISTS follows")
cursor.execute("DROP TABLE IF EXISTS comments")
cursor.execute("DROP TABLE IF EXISTS photos")
cursor.execute("DROP TABLE IF EXISTS likes")
cursor.execute("DROP TABLE IF EXISTS photo_tags")
cursor.execute("DROP TABLE IF EXISTS tags")
cursor.execute("DROP TABLE IF EXISTS users")

# follow table
create_follows_table = """
CREATE TABLE follows (
    follower_id INT,
    followee_id INT,
    created_at DATETIME,
    PRIMARY KEY (follower_id, followee_id)
)
"""
cursor.execute(create_follows_table)

# comments table
create_comments_table = """
CREATE TABLE comments (
    id INT PRIMARY KEY,
    comment_text TEXT,
    user_id INT,
    photo_id INT,
    created_at DATETIME
)
"""
cursor.execute(create_comments_table)

# photos table
create_photos_table = """
CREATE TABLE photos (
    id INT PRIMARY KEY,
    image_url VARCHAR(255),
    user_id INT,
    created_date DATE
)
"""
cursor.execute(create_photos_table)

# likes table
create_likes_table = """
CREATE TABLE likes (
    user_id INT,
    photo_id INT,
    created_at DATETIME,
    PRIMARY KEY (user_id, photo_id)
)
"""
cursor.execute(create_likes_table)

# photo_tags table
create_photo_tags_table = """
CREATE TABLE photo_tags (
    photo_id INT,
    tag_id INT,
    PRIMARY KEY (photo_id, tag_id)
)
"""
cursor.execute(create_photo_tags_table)

# tags table
create_tags_table = """
CREATE TABLE tags (
    id INT PRIMARY KEY,
    tag_name VARCHAR(255),
    created_at DATETIME
)
"""
cursor.execute(create_tags_table)

# users table
create_users_table = """
CREATE TABLE users (
    id INT PRIMARY KEY,
    username VARCHAR(255),
    created_at DATETIME
)
"""
cursor.execute(create_users_table)

# Reading the cleaned dataset from CSV files
follows_df = pd.read_csv('follows_cleaned.csv')
comments_df = pd.read_csv('comments_cleaned.csv')
photos_df = pd.read_csv('photos_cleaned.csv')
likes_df = pd.read_csv('likes_cleaned.csv')
photo_tags_df = pd.read_csv('photo_tags_cleaned.csv')
tags_df = pd.read_csv('tags_cleaned.csv')
users_df = pd.read_csv('users_cleaned.csv')

# Converting datetime columns to the correct format
def parse_datetime(dt_str):
    try:
        return datetime.strptime(dt_str, '%d-%m-%Y %H:%M')
    except ValueError:
        return None

follows_df['created_at'] = follows_df['created_at'].apply(parse_datetime)
comments_df['created_at'] = comments_df['created_at'].apply(parse_datetime)
photos_df['created_date'] = photos_df['created_date'].apply(parse_datetime)
likes_df['created_at'] = likes_df['created_at'].apply(parse_datetime)
users_df['created_at'] = users_df['created_at'].apply(parse_datetime)
tags_df['created_at'] = tags_df['created_at'].apply(parse_datetime)

# Inserting data into all the tables

#follow table
insert_follows_data = """
INSERT INTO follows (follower_id, followee_id, created_at)
VALUES (%s, %s, %s)
"""
follows_data = follows_df[['follower_id', 'followee_id', 'created_at']].values.tolist()
cursor.executemany(insert_follows_data, follows_data)

#comments table
insert_comments_data = """
INSERT INTO comments (id, comment_text, user_id, photo_id, created_at)
VALUES (%s, %s, %s, %s, %s)
"""
comments_data = comments_df[['id', 'comment_text', 'user_id', 'photo_id', 'created_at']].values.tolist()
cursor.executemany(insert_comments_data, comments_data)

#photos table
insert_photos_data = """
INSERT INTO photos (id, image_url, user_id, created_date)
VALUES (%s, %s, %s, %s)
"""
photos_data = photos_df[['id', 'image_url', 'user_id', 'created_date']].values.tolist()
cursor.executemany(insert_photos_data, photos_data)

# likes table
insert_likes_data = """
INSERT INTO likes (user_id, photo_id, created_at)
VALUES (%s, %s, %s)
"""
likes_data = likes_df[['user_id', 'photo_id', 'created_at']].values.tolist()
cursor.executemany(insert_likes_data, likes_data)

#photo_tags table
insert_photo_tags_data = """
INSERT INTO photo_tags (photo_id, tag_id)
VALUES (%s, %s)
"""
photo_tags_data = photo_tags_df[['photo_id', 'tag_id']].values.tolist()
cursor.executemany(insert_photo_tags_data, photo_tags_data)

#tags table
insert_tags_data = """
INSERT INTO tags (id, tag_name, created_at)
VALUES (%s, %s, %s)
"""
tags_data = tags_df[['id', 'tag_name', 'created_at']].values.tolist()
cursor.executemany(insert_tags_data, tags_data)

# users table
insert_users_data = """
INSERT INTO users (id, username, created_at)
VALUES (%s, %s, %s)
"""
users_data = users_df[['id', 'username', 'created_at']].values.tolist()
cursor.executemany(insert_users_data, users_data)

# Commit the changes and close the connection
cnx.commit()
cursor.close()
cnx.close()
